# Analyzing racial disparities in SF traffic stops

To start getting our hands dirty with learning data science, we're going to explore San Francisco's traffic stops. Our goals today are twofold:
1. To learn how to use data science to test for racial discrimination, and to investigate whether there is racial discrimination in our SF data. 
2. To learn a bit of `R`, so that tomorrow you can all start doing your own analysis on different datasets!

## Getting started 

First, let's load the necessary libraries and data that will allow us to begin our investigation!

In [ ]:
# Some initial setup
options(digits = 3)
library(ggplot2)
library(dplyr)
library(readr)
library(magrittr)
library(lubridate)
library(tidyr)
theme_set(theme_bw())
getwd()

# Read the data
stops <- read_rds("opp_data/san_francisco_stop_data.rds")
pop_2015 <- read_rds("opp_data/sf_pop_2015.rds")

### Covering the basics

The core of `R` is the dataframe. We've given you one to start with, in the form
of `stops`. Think of dataframes like a spreadsheet: they have rows and columns.
Usually, rows are a "datapoint": in `stops`, each row corresponds to a single
stop from San Francisco. The columns are the "variables": again, in `stops`,
these are the things we know about the stop, like where the stop happened, the
age of the driver, whether an arrest was made, and so on.

We can take a peak simply by typing `stops` into an R chunk:

In [ ]:
stops

### Functions

In math, functions are a way to "do something" to an input. So `f(a)=b` takes a number `a`, and applies `f()`, and gets the output `b`. In programming, we also have functions! Most of the functions we'll use allow us to manipulate our dataframe as the input. 

So if we want to find the number of rows in our dataframe, we'd use the function `nrow()`, which takes a dataframe (like `stops`) as an input, and then outputs an integer (the number of rows in `stops`).

In [ ]:
nrow(stops)

### Your turn

To find the number of columns, we (unsurprisingly) use `ncol`. Try it!

In [ ]:
# Find the number of columns in `stops`


To figure out what the names of our columns are, we can use `colnames()`.

In [ ]:
# Find the column names in `stops`


Now, if we want to take a peak at our dataframe without printing the whole 900,000 x 13 table, we can use the functions `head()` or `tail()` to see the first few or last few rows.

In [ ]:
head(stops)

**Pro-tip:** If you're ever confused about a function and want to know more about it, what it does, how to use it, etc., every function has "documentation" to help! To know more about the `head()` function, simply run a code chunk with `?head`. It provides way more information than you might want or need -- but if you scroll down to the "Examples" section, those usually help!

## Exercise 1: Stop dates

For this first exercise, let's get a better sense of what time range our `stops` data covers. To do this, we'll be dealing with the `date` column in our dataframe. 

1. What happens when you run `stops$date`? How about `pull(stops, date)`? What do `$` and `pull()` do?

2. What date range does our dataset cover? (Hint: Try exploring the `min()` and `max()` functions, or the `range()` function!)

In [ ]:
## EXERCISE 1: YOUR CODE HERE


Take a look at the two versions of the code below. They do the same thing. See if you can understand what's going on in the second one -- what does `%>%` seem to be doing?

In [ ]:
# Confirm that these give the same answer:

# Method 1: nested
range(pull(stops, date))

# Method 2: multi-line
stops %>% 
    pull(date) %>% 
    range()


**tidyverse tip**: The second method uses a funky symbol, `%>%` called the "pipe", which is the crux of the tidyverse. The pipe helps to keep our code clean. It allows us to read top-down rather than inside-out (which is what method 1 above requires of us). Each line simply applies to the result of the previous line:
* We start with `stops`,
* then we apply `pull(date)` to the above (stops), getting us a list of dates,
* then we apply `range()` to the above (a list of dates).

More formally, the pipe operator
just places the previous item into the first argument of the function. So,
`x %>% f()` is simply `f(x)`. While in a one-function call the pipe might feel
silly and unnecessary, it's going to become _really_ helpful once we start
wanting to do multiple transformations to our data. 

## Preparing our data

For some of our analysis, we'll want to focus on the most recent full year: 2015.

To do this we'll want to use the _year_ of each stop, but _year_ isn't currently a column in our dataset. Let's add it!

**tidyverse function: `mutate()`**

We can use the `mutate()` function to fix add a `yr` column to `stops`.
The `mutate()` function adds new columns to a dataframe based on old columns.
The basic setup is `mutate(DATA, NEW_COL = FUN(OLD_COL))` where 
* `DATA` is our
dataframe, 
* `NEW_COL` is the name of the new column we want, and 
* `FUN` is the function we apply to the old column, `OLD_COL`, to get it.

### You try!

In the space below:

1. use the `year()` and `mutate()` functions to add a new column called `yr` to our `stops` dataframe, and
2. use the assignment operator `<-` (it's like = in `R`) to create a new variable, `stops_w_yr`.

In [ ]:
# Add a yr column to `stops`


**Note:** When we write code chunks and _don't_ save our result using `<-`, that result does not overwrite or in any way change the data. To change the data, we need to use the process above, creating a new variable, or we could overwrite the original dataframe (`stops <- stops %>% ...` -- but be careful, because you could accidentally overwrite the dataframe with something you didn't expect!)

Now, we can investigate this new `yr` column in a few ways. 
1. We can check it's acutally there by looking at `colnames(stops_w_yr)`.
2. We can compute the range of years using `range(stops_w_yr$yr)`.
3. We can count the number of stops per year: `stops_w_yr %>% count(yr)`. 

### You try

Play around with these! Make sure to try the last one.

In [ ]:
# Investigate your new `yr` column. 
# Make sure to try counting the number of stops per year!


### Discuss

What do you notice about stop counts over the years?

### Back to data prep

Now let's get back to prepping our data. To get to our desired date range of the most recent full year (2015), we will 
1. Use the `filter()` function to specify the years we want, and 
2. Again use the assignment operator `<-` (it's like = in `R`) to create a new variable, `stops_2015`.

**tidyverse function: `filter()`**

* The `filter()` function is used to separate rows from the dataframe that
interest us from rows that do not. 
* In particular, `filter(DATA, CONDITION)`
returns `DATA` with all of the rows that satisfy `CONDITION` removed. 
* For
instance, we might want to only look at stops from 2015. To do this, we would type `stops %>% filter(yr == 2015)`, since we only want
rows from `stops` where the `yr` column is (i.e., `==`) `2015`. 
* We can also filter on multiple conditions, just separating each condition with a comma. So, for example, if we wanted all stops between 2011 and 2015, we would write `stops %>% filter(yr >= 2011, yr <=2015)`.

### Your turn

Create a new variable, `stops_2015` that is our stops dataframe filtered to just those that happened in the year (`yr`) 2015. 

In [ ]:
# Use the filter function to get just stops from 2015


Just to be extra sure, let's check our date range in this new dataframe, `stops_2015`!

In [ ]:
# What date range does stops_2015 cover?


## Exercise 2: Stops by race group

For this second exercise, let's compute the racial breakdown of traffic stops. To do this, we'll need two functions that we've already seen: `count()` and `mutate()`.

1. Count how many stops per race group our `stops_2015` dataset has, saving your result to a new dataframe: `stops_by_race`. 

2. Describe in words what we'd need to do to find the proportion of stops that were of white drivers.

3. To do the above computation for each race group, we can add additional column to `stops_by_race` using the `mutate()` function. Overwrite `stops_by_race`, adding a new column `p` with the proportion of stops that were made of drivers of each race group.

4. Discuss: What do these proportions mean? Are drivers of certain race groups being stopped more than others? What might we be missing to really start interpreting these values?

In [ ]:
# EXERCISE 2: YOUR CODE HERE


## Stop rates

In order to do this baseline comparison, we need to understand the racial
demographics in our SF population data. (Note: This is why we wanted just one full year: comparing the number of stops in a year to the population from that year.) The data as we've given it to you
has raw population numbers from 2015. To make it useful, we'll need to compute the
_proportion_ of SF residents in each demographic group. As before, we do this using the `mutate()` function.

### You try

* Take a look at `pop_2015`, then
* mutate `pop_2015`, adding a column `p` that shows us what proportion of the population is white, black, Hispanic, Asian, and other.

In [ ]:
# Find the racial breakdown of SF's 2015 population


### Discuss

What do the population proportions tell you about the stop proportions we computed before?


### Adding rigor

We can tell a lot just by eyeballing these two sets of proportions. But let's be a bit more
rigorous about this. If we merge the two tables together, we can compute stop 
rates by race group (i.e., number of stops per person). 

**R function: `merge()`**

One way to put tables together is with the `merge()` function. We need to
input three things into this function: 
 1. our main table
 2. the second table we'd like to merge with the first table, and
 3. instructions on how to merge them. 

In this case, the two tables we
want to merge are 
 1. the table of stops counted according to `race`, and
 2. the table of population by race: `pop_2015`. 
 
The instruction for combining the tables is 
 3. to merge rows that give information about the same race groups.

To implement 3., we give `merge()` the argument `by =
"race"`. This means that `merge()` will 
 * look at the first table---
i.e., the table stops counted by race---and go to the `race` column
in each row.
 * Then, `merge()` will take what it finds there---in this case,
`"asian/pacific islander"`, `"black"`, `"hispanic"`, `"other/unknown"`, and
`"white"`---and look in the second table, i.e., `pop_2015`, for all the
rows that contain the same information in `pop_2015`'s race column.
 * Finally,
it will add the second row at the end of the first to create a new row with
information from both. 

What we end up with is a dataframe with all of the
columns from _both_ tables.

The process is a little complicated, and we won't use it too much, so don't
worry if the abstract description doesn't make sense. To get a better
understanding of what's going on, try merging the two tables described above,
being sure to include the `by = "race"` argument.


## Exercise 3: computing stop rates by race group

1. First, merge together `stops_by_race` and `pop_2015` by "race", using the `merge()` function. Name your result `stops_and_pop_by_race`.
2. Add a column `stop_rate` to `stops_and_pop_by_race`, that is simply the number of stops divided by the number of people. (Hint: the `mutate()` function will be helpful!)
3. Now we can divide the black (or Asian, or Hispanic, or "other") stop rate by the white stop rate to be able to make 
a quantitative statement about how much more often black drivers are stopped compared to white drivers, relative to their share of the city's population. Using `R` as a calculator, do this!
4. Discuss your results.

In [ ]:
# EXERCISE 3: YOUR CODE HERE


### Thought exercise: where stop rates fall short

While these baseline stats give us a sense that there are racial disparities in
policing practices in SF, they are not strong evidence of discrimination. The
argument against using stop rates (often called "benchmarking" or the "benchmark test") is that we haven't identified the correct
baseline to compare to. 
* Why isn't population the best thing to compare to (i.e., the best denominator of our stop rate)?
* What would the ideal denominator of our stop rate be?
* What other baselines (denominators) could we use? Are any of these ideal?

In [ ]:
# Your thoughts here (or just discuss)


## Searches

Let's next consider how often drivers of different race groups were searched. Computing search rates is actually easier than stop rates because we don't need an external population benchmark.
We can use the stopped population as our baseline, defining search rate to be the proportion of stopped people who were subsequently searched. 

**tidyverse functions: `group_by()` and `summarize()`**

One thing that we often want to do with data is disaggregate it. That is, we
might want to take the data and break it down into smaller subpopulations. Then,
when we ask questions, we can ask about each piece---for instance, each
demographic group, or each police district---instead of asking about the population as a whole.

The way to do this in `R` is with `group_by()` and `summarize()`. The standard way
to use `group_by()` is to call `group_by(DATA, COL_NAME)`, where 
* `DATA` is our dataframe and 
* `COL_NAME` is the name of a column. 
What `group_by()` then does is
take all the rows in the dataframe `DATA` and put them into different groups,
one for each different value in the column `COL_NAME`. So, for instance, if we
called `group_by(stops_w_yr, district)`, `R` would hand back to us the `stops_w_yr`
dataframe with all of its columns broken into different groups, one for each
police district. (Note: At this stage, the dataframe doesn't _look_ any different to the human eye, since the groupings are happening behind the scenes.)

Try it below!

In [ ]:
stops_w_yr %>%
    group_by(district)

The second step is to do something with those groups. That's what `summarize()`
does. The way `summarize()` works is to take a dataframe broken into groups by
`group_by()` and calculate a statistic for each group. The basic syntax is
`summarize(DATA, STAT = FUN(COL_NAME))`, where 
* `DATA` is some dataframe broken
up by `group_by()`, 
* `STAT` is some statistic we want to calculate, 
* `FUN` is the
function that calculates that statistic, and 
* `COL_NAME` is the name of the
column (or columns) used to calculate the statistic.

Let's put it all together with a simple example first.

In [ ]:
# Consider the following dataframe (a "tibble" is just a dataframe)
d <- tibble(
    name = c("Heaven", "Jesus", "Grace", "Maria", "Juan", "Tram"), 
    gender = c("f", "m", "f", "f", "m", "f"), 
    color = rep(c("green", "blue"), 3),
    food = c(rep("apple", 3), rep("banana",3)),
    n_items = c(4,10,12,14,7,5)
)
d

In [ ]:
# If we want to figure out how many foods the girls have and how
# many foods the boys have, we can `group_by` gender and then 
# `summarize` by summing the number of items.
d %>%
    group_by(gender) %>%
    summarize(total_items = sum(n_items))

In [ ]:
# We can also compute the _average_ number of items, the same way!
d %>%
    group_by(gender) %>%
    summarize(
        total_items = sum(n_items),
        avg_items = mean(n_items)
    )

In [ ]:
# We can use the function `n()` to tell us how many instances
# exist in each group (i.e., how many boys and how many girls we have)
# This is another way we could compute the average ourselves!
d %>%
    group_by(gender) %>%
    summarize(
        total_items = sum(n_items),
        num_people = n(),
        avg_items = total_items / num_people
    )


Make sense? Now, using `group_by()` and `summarize()`, we'll calculate search rates by race group.

## Exercise 4: Search rates

1. Compute search rates by race group. (Hint: Think about what information you'd need to compute a search rate. The `n()` function might be helpful!)
2. Discuss the search rate findings. Are some race groups searched more often than other race groups, relative to their share of stopped drivers?

NOTE: Since we're not comparing to population numbers, we can return to using our full `stops_w_yr` dataset, with all years!

In [ ]:
# YOUR CODE HERE


## Thought exercise: where search rates fall short
* Do search rates have similar issues as we found with stop rates? Why or why not?
* What might "justifiably" lead search rates to differ by race group?

In [ ]:
# Write your thoughts here (or just discuss)!


## Outcome test

To circumvent the benchmarking problem, it's common to turn to the search 
decision, rather than the stop decision. This is because we have a notion of
what a "successful" search is. The legal justification for performing a search
is probable cause that the driver possesses contraband. So a successful search
is one which uncovers contraband.

We thus turn to rates of successful searches. That is, what proportion of
searches, by race, were successful? This proportion is known as the contraband
recovery rate, or the "hit rate." If racial groups have different hit rates, it
can imply that racial groups are being subjected to different standards.

## Thought Exercise: Hit rate interpretation

As a caricatured example, suppose among white drivers who were searched, 
officers found contraband 99% of the time, while among black drivers who were
searched, officers found contraband only 1% of the time. 
* Is this police department's search policy discriminatory? 
* Why or why not?
* In general how can we use hit rates to understand whether a search policy is discriminatory?

In [ ]:
# Write your thoughts here (or just discuss)!



Next let's investigate a non-caricatured case: real hit rates by race group in SF.

## Exercise 5: Hit rates

1. Filter to drivers who were searched, and then compute the hit rate (rate of contraband recovery) by race group. Remember your `group_by()` and `summarize()` functions!

2. Discuss your findings. 

In [ ]:
# YOUR CODE HERE


What if hit rates vary by police district? If the bar for stopping
people, irrespective of race, is lower in certain police districts, and black
individuals are more likely to live in neighborhoods in those districts, then
the observed disparities may not reflect bias.

Let's compute hit rates by race _and_ district. We can do this simply by adding multiple arguments to the `group_by()` function. Run the code below.

In [ ]:
hit_rates <- 
  stops_w_yr %>% 
  filter(searched) %>% 
  group_by(race, district) %>% 
  summarize(hit_rate = mean(contraband_found))

hit_rates %>% nrow()

This is too many hit rates to compare in one table!

## Exercise 6: Visualization brainstorm

Sketch out using pen and paper how you might try to use visualizations to help us synthesize the 50 hit rates above. Start with the question we're trying to answer (Are hit rates for minority drivers lower than hit rates for white drivers?) -- and then think about what type of plot might best help you answer that question. See if you can come up with at least 3 different sketches!

## One way to visualize: scatterplots

In [ ]:
# Reshape table to show hit rates of minorities vs white drivers
reshaped_hit_rates <-
  hit_rates %>% 
  spread(race, hit_rate, fill = 0) %>% 
  rename(white_hit_rate = white) %>% 
  gather(
      minority_race, minority_hit_rate, 
      c(black, hispanic, `asian/pacific islander`, other)
  ) %>%
  arrange(district)

# We'll use this just to make our axes' limits nice and even
max_hit_rate <-
  reshaped_hit_rates %>% 
  select(ends_with("hit_rate")) %>% 
  max()

# Get corresponding number of searches (to size points).
# Again, for each district we want to know the number of white+black searches
# and white+Hispanic searches. This requires the same spreading and gathering
# as our previous data-munging.
search_counts <-
  stops_w_yr %>% 
  filter(searched) %>%  
  count(district, race) %>% 
  spread(race, n, fill = 0) %>% 
  rename(num_white_searches = white) %>% 
  gather(
      minority_race, num_minority_searches, 
      c(black, hispanic, `asian/pacific islander`, other)
  ) %>% 
  mutate(num_searches = num_minority_searches + num_white_searches) %>% 
  select(district, minority_race, num_searches)

# Now let's plot!
reshaped_hit_rates %>% 
  left_join(
    search_counts, 
    by = c("district", "minority_race")
  ) %>% 
  ggplot(aes(
    x = white_hit_rate,
    y = minority_hit_rate
  )) +
  geom_point(aes(size = num_searches), pch = 21) +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed") +
  scale_x_continuous("White hit rate", 
    limits = c(0, max_hit_rate + 0.01),
    labels = scales::percent_format(accuracy = 1)
  ) +
  scale_y_continuous("Minority hit rate", 
    limits = c(0, max_hit_rate + 0.01),
    labels = scales::percent_format(accuracy = 1)
  ) +
  coord_fixed() +
  facet_wrap(vars(minority_race))

## Exercise 7: Plot interpretation

Explain what you see above. What does each point represent? What does the dotted line represent? What do these plots tell us about discrimination in search practices?